# <span style="color:darkblue"> Assignment 2: Formula 1 Analysis </span>

<font size = 4>
By: Tanya Jagdish

## <span style="color:darkblue"> Context </span>

## <span style = "color:darkblue"> Import libraries and loading datasets </span>

In [ ]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

: 

In [ ]:
#import necessary datasets

constructors = pd.read_csv("data_raw/constructors.csv")
constructor_results = pd.read_csv("data_raw/constructor_results.csv")   
races = pd.read_csv("data_raw/races.csv")
drivers = pd.read_csv("data_raw/drivers.csv")
qual = pd.read_csv("data_raw/qualifying.csv")


: 

<font size = 3>
Note on dataset selection for this analysis

<font size = 2>

## <span style = "color:darkblue" > Top 3 constructors: 1981-2020, 2001-2020

### <span> Total points per season

<font size = 2>

- Create a table to help calculate total points per constructor in each season/year

- Since the constructor_results table does not have a year column, we add one by merging it with the races table, using a common identifier as shown below

- This table will be helpful in calculating top 3 and average constructor score across the required time periods

In [ ]:
#data checks before merging

#check for dtypes in both datasets
print(constructor_results['raceId'].dtypes)
print(races['raceId'].dtypes)


#Data length checks
print("")
print("constructor_results dataset")
print(len(constructor_results))
print(len(constructor_results['raceId'].unique()))
print(len(constructor_results['constructorId'].unique()))

print("")
print("races")
print(len(races['raceId'].unique()))

#note: races dataset has more unique raceId values than the constructor_results dataset
#we will do a left merge to retain all values from constructor_results dataset

int64
int64

constructor_results dataset
12170
1016
175

races
1102


: 

In [ ]:
#create a seasons total points table for each constructor in each year

constructor_results_merged = pd.merge(constructor_results,
                                      races[['raceId', 'year']], #add year into constructor results table
                                        on='raceId',
                                        how='left')


#check for number of rows in the new table + data integrity
print("")
print(len(constructor_results_merged))
print(len(constructor_results_merged['raceId'].unique()))
print(len(constructor_results_merged['constructorId'].unique()))


12170
1016
175


: 

In [ ]:
#create season_totals table to calculate total points per constructor per year

season_totals = (
    constructor_results_merged
    .groupby(['year', 'constructorId'],       #group by year and constructorId, summing points
             as_index = False)['points']
             .sum() 
    .rename(columns={'points':'total_points'}) #rename points column to total_points
)

#check for number of rows in the new table
print("")
print(len(season_totals))
print(len(constructor_results_merged['raceId'].unique()))
print(len(constructor_results_merged['constructorId'].unique()))

#check for lowest and highest year
print("")
print(season_totals['year'].min())
print(season_totals['year'].max())


929
1016
175

1956
2022


: 

In [65]:
#Add constructor names into season_totals
season_totals = season_totals.merge(
    constructors[['constructorId', 'name']], 
    on ='constructorId', 
    how ='left') 

#preview first few rows to make sure the merge worked
display(season_totals.head())

#check for duplicate years for each constructor
print("")
print(len(season_totals['year'].unique()))
print(len(season_totals['constructorId'].unique()))


MergeError: Passing 'suffixes' which cause duplicate columns {'name_x'} is not allowed.

In [62]:
#getting to know the merged dataset

#highest ever points scored by a constructor in a season
sorted_season_totals = season_totals.sort_values(by='total_points', ascending=False)
sorted_season_totals.head(10)

,year,constructorId,total_points,name_x,name_y
866,2016,131,765.0,Mercedes,Mercedes
922,2022,9,759.0,Red Bull,Red Bull
896,2019,131,739.0,Mercedes,Mercedes
854,2015,131,703.0,Mercedes,Mercedes
843,2014,131,701.0,Mercedes,Mercedes
877,2017,131,668.0,Mercedes,Mercedes
887,2018,131,655.0,Mercedes,Mercedes
806,2011,9,650.0,Red Bull,Red Bull
915,2021,131,613.5,Mercedes,Mercedes
829,2013,9,596.0,Red Bull,Red Bull


<font size = 5>
Part (a), (b) and (c)


<font size = 3>
- Calculating Top 3 constructors across the following time periods: 1981 - 2020, 2001 - 2020


In [ ]:
#creating helper dataframes for calculations
construct_81_20 = season_totals[(season_totals['year'] >= 1981) & (season_totals['year'] <= 2020)]
construct_01_20 = season_totals[(season_totals['year'] >= 2001) & (season_totals['year'] <= 2020)]

#calculating sum of total points for each constructor over the specified periods and then sorting to get top 3
top3_81_20 = (construct_81_20.groupby('name', as_index = False)['total_points']
               .sum()
               .sort_values(by='total_points', ascending = False)
               .head(3)
              )

top3_01_20 = (construct_01_20.groupby('name', as_index = False)['total_points']
                .sum()
                .sort_values(by='total_points', ascending = False)
                .head(3)
                  )

display(top3_81_20)
display(top3_01_20)

<font size = 3>
- Calculating average points across constructors in the following time periods: 1981-2020, 2001-2020

In [ ]:
#calculating total points for each constructor over the specified periods
totalpoints_81_20 = (construct_81_20
                     .groupby(["constructorId", "name"], as_index = False)["total_points"]
                     .sum()
                    )
totalpoints_01_20 = (construct_01_20
                     .groupby(["constructorId", "name"], as_index = False)["total_points"]
                     .sum()
                    )

#calculating average total points across all constructors over the specified periods
avg_81_20 = totalpoints_81_20["total_points"].mean()
avg_01_20 = totalpoints_01_20["total_points"].mean()

display(avg_81_20)
display(avg_01_20)

<font size = 3>
Analysis comments

- As seen above, the top three constructors between 1981-2020 are: Ferrari, Mercedes, McLaren
- Whereas, the top three constructors between 2001-2020 are: Ferrari, Mercedes, Red Bull
- In the second time period, Red Bull replaced McLaren on the leader board
- The top three scorers had significantly larger scores compared to the average across all constructs
- For instance, the top scores across both time periods scored more than 5000 points
- But the average total points was less than 1000 in both time periods


<font size = 5>
(d) and (e) Ferrari analysis (1981-2020)